In [1]:
#Load dataset
import numpy as np
X_train=[]
X_validation=[]
X_test=[]
Y_train=[]
Y_validation=[]
Y_test=[]
with open('Train1.npy', 'rb') as f:
    X_train = np.load(f)
    Y_train = np.load(f)
with open('Validation1.npy', 'rb') as f1:
    X_validation = np.load(f1)
    Y_validation = np.load(f1)
with open('Test1.npy', 'rb') as f2:
    X_test = np.load(f2)
    Y_test = np.load(f2)


In [2]:
# Import DNN model from file
from keras.models import load_model
import tensorflow as tf
model = tf.keras.models.load_model('DNN.h5')

Using TensorFlow backend.


In [3]:
# Import RF and KNC models from file
import joblib
model1 = joblib.load('RF.sav')
model2 = joblib.load('KNC.sav')

In [6]:
# Testing with 50 random samples from test dataset
from scipy import stats
fl = open("Testing.txt", "w")
fl.write("Number,\tDNNbrP,\tRFbrP,\tKNCbrP,\tTTS1,\tTTP1,\tTTS2,\tTTP2,\tTTS3,\tTTP3\n")
for m in range(50):
    ListOfSamples=[]
    ListOfIndexes=[]
    for j in range(1000):
        ind=np.random.randint(0,1699)
        ListOfSamples.append(X_test[ind])
        ListOfIndexes.append(Y_test[ind])
    ListOfSamples=np.array(ListOfSamples)
    ModPred=np.argmax(model.predict(ListOfSamples),axis=1)
    ModPred=np.array(ModPred)
    ModBin=[]
    MBP=0
    for i in range (len(ListOfIndexes)):
        if (ListOfIndexes[i]==ModPred[i]):
            ModBin.append(0)
            MBP+=1
        else:
            ModBin.append(1)
    ModPred1=model1.predict(ListOfSamples)
    ModBin1=[]
    MBP1=0
    for i in range (len(ListOfIndexes)):
        if (ListOfIndexes[i]==ModPred1[i]):
            ModBin1.append(0)
            MBP1+=1
        else:
            ModBin1.append(1)
    ModPred2=model2.predict(ListOfSamples)
    ModBin2=[]
    MBP2=0
    for i in range (len(ListOfIndexes)):
        if (ListOfIndexes[i]==ModPred2[i]):
            ModBin2.append(0)
            MBP2+=1
        else:
            ModBin2.append(1)
    T1,tp1=stats.ttest_rel(ModBin2,ModBin)
    T2,tp2=stats.ttest_rel(ModBin2,ModBin1)
    T3,tp3=stats.ttest_rel(ModBin1,ModBin)
    fl.write(str(m)+',\t'+str(MBP)+',\t'+str(MBP1)+',\t'+str(MBP2)+',\t'+str(T1)+',\t'+str(tp1)+',\t'+str(T2)+',\t'+str(tp2)+',\t'+str(T3)
        +',\t'+str(tp3)+'\n')
    print("i",m)
fl.close()

i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
i 12
i 13
i 14
i 15
i 16
i 17
i 18
i 19
i 20
i 21
i 22
i 23
i 24
i 25
i 26
i 27
i 28
i 29
i 30
i 31
i 32
i 33
i 34
i 35
i 36
i 37
i 38
i 39
i 40
i 41
i 42
i 43
i 44
i 45
i 46
i 47
i 48
i 49


In [8]:
# Confusion matrix create and test
from sklearn.metrics import confusion_matrix
yp=model.predict(X_test)
Pt1=np.argmax(yp,axis=1)
cm1=confusion_matrix(Y_test,Pt1)
Pt2=model1.predict(X_test)
cm2=confusion_matrix(Y_test,Pt2)
Pt3=model2.predict(X_test)
cm3=confusion_matrix(Y_test,Pt3)
print(cm3)

[[ 57   0   9   8  62  11   1   0  15   7]
 [  4 121   0  11  15   2   5   1   4   7]
 [ 21   0  10   6 111  20   0   0   2   0]
 [  1   0   0 145  13   5   0   0   6   0]
 [  3   0   0   4 160   2   0   0   0   1]
 [  9   0   0  18  81  53   0   1   5   3]
 [ 16   0  19   9  56   3  18   2   1  46]
 [ 11   0  23   5  85   5   0  23   1  17]
 [  0   0   0  76   0  19   0   0  75   0]
 [  2   0   0  51  74   1   6   0  10  26]]
